In [24]:
from dotenv import load_dotenv
import os
import openai

# Charge les variables du fichier .env
load_dotenv()

# Configuration Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

DEPLOYMENT_NAME = os.getenv("AZURE_MODEL_DEPLOYMENT")
EMBEDDING_NAME = os.getenv("AZURE_EMBEDDING_DEPLOYMENT")


In [25]:
response = openai.ChatCompletion.create(
    engine=DEPLOYMENT_NAME,
    messages=[
        {"role": "user", "content": "Test depuis Azure OpenAI, est-ce que tu me vois ?"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Oui, je te vois ! 😊 Je suis là pour répondre à tes questions et t'aider. Que puis-je faire pour toi aujourd'hui ?


In [26]:
embedding = openai.Embedding.create(
    input="ma mère s'appelle Nadeige",
    engine=EMBEDDING_NAME
)

print(f"Longueur vecteur : {len(embedding['data'][0]['embedding'])}")


Longueur vecteur : 1536


# RAG


In [27]:
def chunk_text(text, max_tokens=500):
    import tiktoken
    enc = tiktoken.get_encoding("cl100k_base")
    paragraphs = text.split("\n\n")
    chunks = []
    current = ""
    for para in paragraphs:
        if len(enc.encode(current + para)) > max_tokens:
            chunks.append(current.strip())
            current = para
        else:
            current += "\n\n" + para
    chunks.append(current.strip())
    return chunks

with open("docs/rules.txt", "r", encoding="utf-8") as f:
    doc = f.read()

chunks = chunk_text(doc)
print(f"Nombre de chunks : {len(chunks)}")


Nombre de chunks : 1


In [28]:
chunk_vectors = []
for chunk in chunks:
    res = openai.Embedding.create(input=chunk, engine=EMBEDDING_NAME)
    vec = res["data"][0]["embedding"]
    chunk_vectors.append((chunk, vec))


In [29]:
import numpy as np

def cosine_sim(v1, v2):
    a, b = np.array(v1), np.array(v2)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def retrieve_relevant_chunks(question, top_k=3):
    q_vec = openai.Embedding.create(input=question, engine=EMBEDDING_NAME)["data"][0]["embedding"]
    ranked = sorted(chunk_vectors, key=lambda x: cosine_sim(x[1], q_vec), reverse=True)
    return [c[0] for c in ranked[:top_k]]


In [30]:
def build_prompt(chunks, question):
    context = "\n\n".join(chunks)
    return f"""
Tu es un assistant qui répond uniquement avec les informations suivantes. Si l'information n'y est pas, dis-le clairement.

CONNAISSANCES :
{context}

QUESTION :
{question}

RÉPONSE :
"""

def ask_rag(question):
    top_chunks = retrieve_relevant_chunks(question)
    prompt = build_prompt(top_chunks, question)
    response = openai.ChatCompletion.create(
        engine=DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()


In [31]:
ask_rag("Comment s'appelle la mère d'Hugo ?")


'La mère d\'Hugo s\'appelle "développe ton racisme".'